In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(root = "data", train = True, download = True, transform = ToTensor())

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




7.4%

100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [5]:
test_data = datasets.FashionMNIST(root = "data", train = False, download = True, transform = ToTensor())

In [13]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

In [16]:
for X, y in test_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64


In [19]:
print(len(train_dataloader))
print(len(test_dataloader))

938
157


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512), nn.ReLU(), nn.Linear(512, 512), nn.ReLU(), nn.Linear(512, 10))
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [25]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [27]:
def train(dataloader, model , loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss = {loss}, current : {current/size}")
            
    
    

In [28]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [29]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss = 2.3065361976623535, current : 0.0010666666666666667
loss = 2.2924880981445312, current : 0.10773333333333333
loss = 2.2766740322113037, current : 0.2144
loss = 2.275527000427246, current : 0.32106666666666667
loss = 2.25191068649292, current : 0.42773333333333335
loss = 2.2223007678985596, current : 0.5344
loss = 2.2299280166625977, current : 0.6410666666666667
loss = 2.197195053100586, current : 0.7477333333333334
loss = 2.198263168334961, current : 0.8544
loss = 2.161410331726074, current : 0.9610666666666666
Test Error: 
 Accuracy: 45.9%, Avg loss: 2.164625 

Epoch 2
-------------------------------
loss = 2.1724672317504883, current : 0.0010666666666666667
loss = 2.1701297760009766, current : 0.10773333333333333
loss = 2.111342191696167, current : 0.2144
loss = 2.1393117904663086, current : 0.32106666666666667
loss = 2.0853524208068848, current : 0.42773333333333335
loss = 2.0120596885681152, current : 0.5344
loss = 2.0503947734832764, 

In [30]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [32]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"), strict = False)

<All keys matched successfully>

In [33]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [34]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
